In [1]:
def start_docker(step):
    global logging
    volumes = ['/wsefs:/wsefs',]
    docker_client = docker.from_env()
    cmd = f'python3 {step}.py'
    docker_image = step.lower()
    logging.info(f'start_docker:{docker_image}')
    container_name = step
    #container_obj = docker_client.containers.run(docker_image, cmd, detach=True, name=container_name)
    container_obj = docker_client.containers.run(docker_image, cmd, detach=False,
                                                 volumes=volumes, auto_remove=True, name=container_name)
    logging.info(f'{cmd} started {container_name} {docker_image}')


In [2]:
import time
import os
import subprocess
import re

import yaml
import json
import docker

import pandas as pd

In [3]:
# python3 -m pip install -U PyYAML

def return_cpu_load():
    cpu_load = [x / os.cpu_count() * 100 for x in os.getloadavg()][-1]
    return cpu_load


def _return_mem_stat():
    # Memory usage
    total_ram = subprocess.run(['free', '-h'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    used_free_shared_buf_avail = total_ram.split('\n')[1]

    return  used_free_shared_buf_avail

def return_available_memory():
    used_free_shared_buf_avail = _return_mem_stat()
    a = re.split('\s+', used_free_shared_buf_avail)
    available_memory = a[3].split('G')[0]
    return float(available_memory)

In [4]:
def make_cmd(plot_id, year, x, y, project_id):
    arguments = f'--year {year} --x {x} --y {y} --plot_id {plot_id} {project_id}'

    return f'python3 ts_user.py {arguments}'


In [5]:
class Tso:
    def __init__(self, csv_file='lcnext_srs5000_final.csv'):
        print('Tso Initailized', csv_file)
        self.csv_file = csv_file
        self.client = docker.from_env()

    def __repr__(self):
        return('REPR Goes Here more to come ©')

    def MAIN_TS_runner(self, start_year, end_year):
         for my_year in range(start_year, end_year+1):
            self._event_loop(my_year)

    def _start_container(self, docker_image, docker_full_cmd, name):
        #container = self.client.containers.run(docker_image, docker_full_cmd, detach=True, auto_remove=True, name=name)
        volumes = ['/efs:/efs',]
        container = self.client.containers.run(docker_image, docker_full_cmd, detach=True, 
                                               volumes=volumes, auto_remove=True, name=name)
        print ( "CONTAINER is ", container.name)
        return(container)


    def _do_job(self, plot_id, my_year, x, y, project_id):

        cmd = make_cmd(plot_id, my_year, x, y, project_id)
        
        print(cmd)
        docker_image =  "chipper-image"
        print(docker_image)
        number_str = str(plot_id).zfill(4)
        job_name=f"{my_year}_{number_str}_{project_id}"
        c = self._start_container(docker_image, cmd, job_name)
        print("real name is", c.name)
        print("==="*30)

    def _return_num_containers(self):
        client = self.client
        try:
            running_containers = client.containers.list()
        except:
            running_containers=[]
        return(len(running_containers))


    def _event_loop(self, year_to_process):

        MAX_LOAD_LEVEL = 150
        MIN_MEMORY_AVAILABLE =  2
        MAX_CONCURRENT_CONTAINERS = 8
        SLEEP_TIME=5

        KILL_CNT = 16

        df = pd.read_csv(self.csv_file)
        print(df.head())
        STOP_AT = len(df)
        CNT = 0
        while True:
            print(f'sleeping for {SLEEP_TIME} .... ')
            time.sleep(SLEEP_TIME)

            mem_avail = return_available_memory()
            cpu_load = return_cpu_load()
            num_running_containers = self._return_num_containers()
            print(f'mem={mem_avail}, cpu={cpu_load}, num_containers={num_running_containers}')

            cpu = False
            if (cpu_load < MAX_LOAD_LEVEL):
                print("CPU is FINE")
                cpu=True

            mem = False
            if mem_avail > MIN_MEMORY_AVAILABLE:
                print("MEM is FINE")
                mem = True


            containers = False
            if num_running_containers < MAX_CONCURRENT_CONTAINERS:
                containers = True
            else:
                print("MAX Container Level Reached - Chill :-)")


            if (mem and cpu and containers):
                print("OK to Launch")
                r = df.iloc[[CNT]].to_dict('records')[0]
                #self._start_etm(str(year_to_process), product) ### start mosaic container
                print("my_year", year_to_process)
                my_year = year_to_process
                x = r['x']
                y = r['y']
                plot_id = r['plot_id']
                project_id = r['project_id']
                print(r)
                self._do_job(plot_id, my_year, x, y, project_id)
                KILL_CNT = KILL_CNT -1
                if KILL_CNT <= 0:
                    break
                CNT = CNT + 1
                if CNT > STOP_AT:
                    break
            #END While


# ============================================================
        print(f"EVENT LOOP COMPLETE Process Annuals - {product}")
                                               

In [6]:
tso=Tso()
print(tso)

tso.MAIN_TS_runner(1984,1984)


Tso Initailized lcnext_srs5000_final.csv
REPR Goes Here more to come ©
   plot_id        x        y  project_id  Strata
0        1 -1115430  1759950        1211      99
1        2   272760  1458900        1211      99
2        3  -537870  1936560        1211      99
3        4  -103260  2202720        1211      99
4        5 -1979280  1773390        1211      99
sleeping for 5 .... 
mem=9.5, cpu=0.5, num_containers=0
CPU is FINE
MEM is FINE
OK to Launch
my_year 1984
{'plot_id': 1, 'x': -1115430, 'y': 1759950, 'project_id': 1211, 'Strata': 99}
python3 ts_user.py --year 1984 --x -1115430 --y 1759950 --plot_id 1 1211
chipper-image
CONTAINER is  1984_0001_1211
real name is 1984_0001_1211
sleeping for 5 .... 
mem=9.4, cpu=1.0, num_containers=1
CPU is FINE
MEM is FINE
OK to Launch
my_year 1984
{'plot_id': 2, 'x': 272760, 'y': 1458900, 'project_id': 1211, 'Strata': 99}
python3 ts_user.py --year 1984 --x 272760 --y 1458900 --plot_id 2 1211
chipper-image
CONTAINER is  1984_0002_1211
real name i

NameError: name 'product' is not defined